In [61]:
import pandas as pd
import warnings
import pycountry
import numpy as np


warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
url = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels.csv')
df = pd.read_csv(url)

In [15]:
df

,Entity,Code,Year,"Combined gross enrolment ratio for tertiary education, female","Combined gross enrolment ratio for tertiary education, male","Combined total net enrolment rate, secondary, male","Combined total net enrolment rate, secondary, female","Combined total net enrolment rate, primary, female","Combined total net enrolment rate, primary, male"
0,Afghanistan,AFG,1820,0.0,0.01,0.01000,0.00000,0.00000,0.01000
1,Afghanistan,AFG,1825,0.0,0.01,0.01000,0.00000,0.00000,0.01000
2,Afghanistan,AFG,1830,0.0,0.01,0.01000,0.00000,0.00000,0.01000
3,Afghanistan,AFG,1835,0.0,0.01,0.01000,0.00000,0.00000,0.01000
4,Afghanistan,AFG,1840,0.0,0.01,0.01000,0.00000,0.00000,0.01000
...,...,...,...,...,...,...,...,...,...
9223,Zimbabwe,ZWE,2000,NaN,NaN,44.75129,40.17528,89.00224,88.52073
9224,Zimbabwe,ZWE,2001,NaN,NaN,45.72983,40.98154,92.54698,91.74628
9225,Zimbabwe,ZWE,2002,NaN,NaN,43.78683,38.72148,89.90188,89.31955
9226,Zimbabwe,ZWE,2003,NaN,NaN,39.19233,35.34257,89.42919,88.45444


In [16]:
# Drop the 'Code' column and rename columns as specified
df = df.drop(columns=['Code'])
df.rename(columns={
    'Entity': 'Country Name',
    'Combined gross enrolment ratio for tertiary education, female': 'Tertiary Enrolment (F)',
    'Combined gross enrolment ratio for tertiary education, male': 'Tertiary Enrolment (M)',
    'Combined total net enrolment rate, secondary, male': 'Secondary Enrolment (M)',
    'Combined total net enrolment rate, secondary, female': 'Secondary Enrolment (F)',
    'Combined total net enrolment rate, primary, female': 'Primary Enrolment (F)',
    'Combined total net enrolment rate, primary, male': 'Primary Enrolment (M)'
}, inplace=True)

In [17]:
# Ensure the dataset is filtered correctly for the required years (1980-2021)
filtered_data = df[(df['Year'] >= 1980) & (df['Year'] <= 2021)]

In [18]:
# Define a function to create a complete set of years for interpolation
def complete_years(df, start_year, end_year):
    idx = pd.MultiIndex.from_product([df['Country Name'].unique(), range(start_year, end_year+1)], names=['Country Name', 'Year'])
    df = df.set_index(['Country Name', 'Year']).reindex(idx).reset_index()
    return df

# Apply the function to the filtered data
complete_data = complete_years(filtered_data, 1980, 2021)


In [19]:
interpolated_data_complete = complete_data.groupby('Country Name', group_keys=True).apply(lambda x: x.sort_values('Year').interpolate(method='linear')).reset_index(drop=True)

/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_30329/533028074.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  interpolated_data_complete = complete_data.groupby('Country Name', group_keys=True).apply(lambda x: x.sort_values('Year').interpolate(method='linear')).reset_index(drop=True)


In [20]:
interpolated_data_complete

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M)
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625
...,...,...,...,...,...,...,...,...
9529,Zimbabwe,2017,9.689270,8.005960,48.960040,48.512550,89.429190,88.454440
9530,Zimbabwe,2018,9.689270,8.005960,48.960040,48.512550,89.429190,88.454440
9531,Zimbabwe,2019,9.689270,8.005960,48.960040,48.512550,89.429190,88.454440
9532,Zimbabwe,2020,9.689270,8.005960,48.960040,48.512550,89.429190,88.454440


In [25]:
interpolated_data_complete.isna().sum()

Country Name                  0
Year                          0
Tertiary Enrolment (F)     1675
Tertiary Enrolment (M)     1675
Secondary Enrolment (M)    2387
Secondary Enrolment (F)    2387
Primary Enrolment (F)      1931
Primary Enrolment (M)      1931
dtype: int64

In [26]:
filtered_data = interpolated_data_complete[(interpolated_data_complete['Year'] >= 1980) & (interpolated_data_complete['Year'] <= 2021)]


In [27]:
def complete_years(df, start_year, end_year):
    idx = pd.MultiIndex.from_product([df['Country Name'].unique(), range(start_year, end_year+1)], names=['Country Name', 'Year'])
    df = df.set_index(['Country Name', 'Year']).reindex(idx).reset_index()
    return df
complete_data = complete_years(filtered_data, 1980, 2021)


In [28]:
filled_data = complete_data.groupby('Country Name').apply(lambda x: x.sort_values('Year').fillna(method='ffill').fillna(method='bfill')).reset_index(drop=True)


/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_30329/2084395555.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_data = complete_data.groupby('Country Name').apply(lambda x: x.sort_values('Year').fillna(method='ffill').fillna(method='bfill')).reset_index(drop=True)


In [30]:
filled_data.isna().sum()

Country Name                 0
Year                         0
Tertiary Enrolment (F)     294
Tertiary Enrolment (M)     294
Secondary Enrolment (M)    672
Secondary Enrolment (F)    672
Primary Enrolment (F)      588
Primary Enrolment (M)      588
dtype: int64

In [34]:
filled_data = filled_data.round(2)

In [35]:
filled_data.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels_cleanded.csv',index=False,encoding='latin-1')

In [53]:
url = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels_cleanded.csv')
df=pd.read_csv(url)
df

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M)
0,Afghanistan,1980,0.50,1.86,13.76,3.52,11.00,46.06
1,Afghanistan,1981,0.52,2.15,14.29,3.85,11.21,45.50
2,Afghanistan,1982,0.54,2.44,14.82,4.18,11.42,44.94
3,Afghanistan,1983,0.56,2.73,15.35,4.51,11.62,44.38
4,Afghanistan,1984,0.58,3.02,15.88,4.83,11.83,43.82
...,...,...,...,...,...,...,...,...
9529,Zimbabwe,2017,9.69,8.01,48.96,48.51,89.43,88.45
9530,Zimbabwe,2018,9.69,8.01,48.96,48.51,89.43,88.45
9531,Zimbabwe,2019,9.69,8.01,48.96,48.51,89.43,88.45
9532,Zimbabwe,2020,9.69,8.01,48.96,48.51,89.43,88.45


In [40]:
df.isna().sum()

Country Name                 0
Year                         0
Tertiary Enrolment (F)     294
Tertiary Enrolment (M)     294
Secondary Enrolment (M)    672
Secondary Enrolment (F)    672
Primary Enrolment (F)      588
Primary Enrolment (M)      588
dtype: int64

In [47]:
# Group by 'Country Name' and apply forward fill, backward fill, and interpolation without using lambda
data_grouped = df.groupby('Country Name').transform(lambda x: x.ffill().bfill().interpolate())

# Reattach 'Country Name' and 'Year' columns to the grouped df
data_grouped['Country Name'] = df['Country Name']
data_grouped['Year'] = df['Year']

# Check for any remaining missing values
remaining_missing_values = data_grouped.isnull().sum()
remaining_missing_values


Year                         0
Tertiary Enrolment (F)     294
Tertiary Enrolment (M)     294
Secondary Enrolment (M)    672
Secondary Enrolment (F)    672
Primary Enrolment (F)      588
Primary Enrolment (M)      588
Country Name                 0
dtype: int64

In [57]:
country_region_mapping = {
    "Afghanistan": "Asia",
    "Africa": "Africa",
    "Albania": "Europe",
    "Algeria": "Africa",
    "American Samoa": "Oceania",
    "Angola": "Africa",
    "Antigua and Barbuda": "North America",
    "Argentina": "South America",
    "Armenia": "Asia",
    "Aruba": "North America",
    "Asia": "Asia",
    "Australia": "Oceania",
    "Austria": "Europe",
    "Azerbaijan": "Asia",
    "Bahamas": "North America",
    "Bahrain": "Asia",
    "Bangladesh": "Asia",
    "Barbados": "North America",
    "Belarus": "Europe",
    "Belgium": "Europe",
    "Belize": "North America",
    "Benin": "Africa",
    "Bermuda": "North America",
    "Bhutan": "Asia",
    "Bolivia": "South America",
    "Bosnia and Herzegovina": "Europe",
    "Botswana": "Africa",
    "Brazil": "South America",
    "British Virgin Islands": "North America",
    "Brunei": "Asia",
    "Bulgaria": "Europe",
    "Burkina Faso": "Africa",
    "Burundi": "Africa",
    "Cambodia": "Asia",
    "Cameroon": "Africa",
    "Canada": "North America",
    "Cape Verde": "Africa",
    "Cayman Islands": "North America",
    "Central African Republic": "Africa",
    "Chad": "Africa",
    "Chile": "South America",
    "China": "Asia",
    "Colombia": "South America",
    "Comoros": "Africa",
    "Congo": "Africa",
    "Costa Rica": "North America",
    "Cote d'Ivoire": "Africa",
    "Croatia": "Europe",
    "Cuba": "North America",
    "Curacao": "North America",
    "Cyprus": "Asia",
    "Czechia": "Europe",
    "Democratic Republic of Congo": "Africa",
    "Denmark": "Europe",
    "Djibouti": "Africa",
    "Dominica": "North America",
    "Dominican Republic": "North America",
    "East Asia and Pacific (WB)": "Asia",
    "East Timor": "Asia",
    "Ecuador": "South America",
    "Egypt": "Africa",
    "El Salvador": "North America",
    "Equatorial Guinea": "Africa",
    "Eritrea": "Africa",
    "Estonia": "Europe",
    "Eswatini": "Africa",
    "Ethiopia": "Africa",
    "Europe": "Europe",
    "Europe and Central Asia (WB)": "Europe",
    "European Union (27)": "Europe",
    "Fiji": "Oceania",
    "Finland": "Europe",
    "France": "Europe",
    "French Polynesia": "Oceania",
    "Gabon": "Africa",
    "Gambia": "Africa",
    "Georgia": "Asia",
    "Germany": "Europe",
    "Ghana": "Africa",
    "Gibraltar": "Europe",
    "Greece": "Europe",
    "Grenada": "North America",
    "Guatemala": "North America",
    "Guinea": "Africa",
    "Guinea-Bissau": "Africa",
    "Guyana": "South America",
    "Haiti": "North America",
    "High-income countries": "Various",
    "Honduras": "North America",
    "Hong Kong": "Asia",
    "Hungary": "Europe",
    "Iceland": "Europe",
    "India": "Asia",
    "Indonesia": "Asia",
    "Iran": "Asia",
    "Iraq": "Asia",
    "Ireland": "Europe",
    "Israel": "Asia",
    "Italy": "Europe",
    "Jamaica": "North America",
    "Japan": "Asia",
    "Jordan": "Asia",
    "Kazakhstan": "Asia",
    "Kenya": "Africa",
    "Kiribati": "Oceania",
    "Kuwait": "Asia",
    "Kyrgyzstan": "Asia",
    "Laos": "Asia",
    "Latin America and Caribbean (WB)": "South America",
    "Latvia": "Europe",
    "Lebanon": "Asia",
    "Lesotho": "Africa",
    "Liberia": "Africa",
    "Libya": "Africa",
    "Liechtenstein": "Europe",
    "Lithuania": "Europe",
    "Low-income countries": "Various",
    "Lower-middle-income countries": "Various",
    "Luxembourg": "Europe",
    "Macao": "Asia",
    "Madagascar": "Africa",
    "Malawi": "Africa",
    "Malaysia": "Asia",
    "Maldives": "Asia",
    "Mali": "Africa",
    "Malta": "Europe",
    "Marshall Islands": "Oceania",
    "Mauritania": "Africa",
    "Mauritius": "Africa",
    "Mexico": "North America",
    "Micronesia (country)": "Oceania",
    "Middle East and North Africa (WB)": "Africa",
    "Middle-income countries": "Various",
    "Moldova": "Europe",
    "Mongolia": "Asia",
    "Montenegro": "Europe",
    "Morocco": "Africa",
    "Mozambique": "Africa",
    "Myanmar": "Asia",
    "Namibia": "Africa",
    "Nauru": "Oceania",
    "Nepal": "Asia",
    "Netherlands": "Europe",
    "New Zealand": "Oceania",
    "Nicaragua": "North America",
    "Niger": "Africa",
    "Nigeria": "Africa",
    "North America": "North America",
    "North America (WB)": "North America",
    "North Korea": "Asia",
    "North Macedonia": "Europe",
    "Norway": "Europe",
    "Oceania": "Oceania",
    "Oman": "Asia",
    "Pakistan": "Asia",
    "Palau": "Oceania",
    "Palestine": "Asia",
    "Panama": "North America",
    "Papua New Guinea": "Oceania",
    "Paraguay": "South America",
    "Peru": "South America",
    "Philippines": "Asia",
    "Poland": "Europe",
    "Portugal": "Europe",
    "Puerto Rico": "North America",
    "Qatar": "Asia",
    "Reunion": "Africa",
    "Romania": "Europe",
    "Russia": "Europe",
    "Rwanda": "Africa",
    "Saint Kitts and Nevis": "North America",
    "Saint Lucia": "North America",
    "Saint Vincent and the Grenadines": "North America",
    "Samoa": "Oceania",
    "San Marino": "Europe",
    "Sao Tome and Principe": "Africa",
    "Saudi Arabia": "Asia",
    "Senegal": "Africa",
    "Serbia": "Europe",
    "Seychelles": "Africa",
    "Sierra Leone": "Africa",
    "Singapore": "Asia",
    "Sint Maarten (Dutch part)": "North America",
    "Slovakia": "Europe",
    "Slovenia": "Europe",
    "Solomon Islands": "Oceania",
    "Somalia": "Africa",
    "South Africa": "Africa",
    "South America": "South America",
    "South Asia (WB)": "Asia",
    "South Korea": "Asia",
    "South Sudan": "Africa",
    "Spain": "Europe",
    "Sri Lanka": "Asia",
    "Sub-Saharan Africa (WB)": "Africa",
    "Sudan": "Africa",
    "Suriname": "South America",
    "Sweden": "Europe",
    "Switzerland": "Europe",
    "Syria": "Asia",
    "Taiwan": "Asia",
    "Tajikistan": "Asia",
    "Tanzania": "Africa",
    "Thailand": "Asia",
    "Togo": "Africa",
    "Tonga": "Oceania",
    "Trinidad and Tobago": "North America",
    "Tunisia": "Africa",
    "Turkey": "Asia",
    "Turkmenistan": "Asia",
    "Turks and Caicos Islands": "North America",
    "Tuvalu": "Oceania",
    "Uganda": "Africa",
    "Ukraine": "Europe",
    "United Arab Emirates": "Asia",
    "United Kingdom": "Europe",
    "United States": "North America",
    "Upper-middle-income countries": "Various",
    "Uruguay": "South America",
    "Uzbekistan": "Asia",
    "Vanuatu": "Oceania",
    "Venezuela": "South America",
    "Vietnam": "Asia",
    "World": "Various",
    "Yemen": "Asia",
    "Zambia": "Africa",
    "Zimbabwe": "Africa",
}

def map_country_to_region(country_name):
    return country_region_mapping.get(country_name, "Unknown")

# Example usage: Assuming 'df' is your dataframe and it has a column 'Country Name'
df['Region'] = df['Country Name'].apply(map_country_to_region)

# Display the first few rows to check the regions
df.head()


,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
0,Afghanistan,1980,0.50,1.86,13.76,3.52,11.00,46.06,Asia
1,Afghanistan,1981,0.52,2.15,14.29,3.85,11.21,45.50,Asia
2,Afghanistan,1982,0.54,2.44,14.82,4.18,11.42,44.94,Asia
3,Afghanistan,1983,0.56,2.73,15.35,4.51,11.62,44.38,Asia
4,Afghanistan,1984,0.58,3.02,15.88,4.83,11.83,43.82,Asia


In [63]:
# Function to map country to region using pycountry and existing mapping
def get_region(country_name):
    try:
        country = pycountry.countries.lookup(country_name)
        return country.region
    except LookupError:
        return country_region_mapping.get(country_name, "Unknown")

# If the region column is missing or needs to be verified
if 'Region' not in df.columns or df['Region'].isnull().any():
    df['Region'] = df['Country Name'].apply(get_region)

# Check the unique regions to ensure correctness
print(df['Region'].unique())

# Define numeric columns to fill missing values
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Function to fill missing values with the mean of the region for each year for numeric columns only
def fill_missing_values(group):
    for col in numeric_columns:
        group[col] = group[col].fillna(group[col].mean())
    return group

# Group by 'Region' and 'Year' and fill missing values with the regional mean for that year
df = df.groupby(['Region', 'Year']).apply(fill_missing_values).reset_index(drop=True)

# Group by 'Country Name' and 'Year' and interpolate missing values linearly for numeric columns only
def interpolate_values(group):
    group[numeric_columns] = group[numeric_columns].interpolate(method='linear')
    return group

df = df.groupby('Country Name', group_keys=True).apply(interpolate_values).reset_index(drop=True)


df.head()


['Asia' 'Africa' 'Europe' 'Oceania' 'North America' 'South America'
 'Various']


/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_30329/4161425608.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['Region', 'Year']).apply(fill_missing_values).reset_index(drop=True)
/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_30329/4161425608.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Country Name', group_keys=True).apply(interpolate

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
0,Afghanistan,1980,0.50,1.86,13.76,3.52,11.00,46.06,Asia
1,Afghanistan,1981,0.52,2.15,14.29,3.85,11.21,45.50,Asia
2,Afghanistan,1982,0.54,2.44,14.82,4.18,11.42,44.94,Asia
3,Afghanistan,1983,0.56,2.73,15.35,4.51,11.62,44.38,Asia
4,Afghanistan,1984,0.58,3.02,15.88,4.83,11.83,43.82,Asia


In [71]:
df['Region'].unique()

array(['Asia', 'Africa', 'Europe', 'Oceania', 'North America',
       'South America', 'Various'], dtype=object)

In [64]:
df.isna().sum()

Country Name               0
Year                       0
Tertiary Enrolment (F)     0
Tertiary Enrolment (M)     0
Secondary Enrolment (M)    0
Secondary Enrolment (F)    0
Primary Enrolment (F)      0
Primary Enrolment (M)      0
Region                     0
dtype: int64

In [68]:
df.describe()

,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M)
count,9534.000000,9534.000000,9534.000000,9534.000000,9534.000000,9534.000000,9534.000000
mean,2000.500000,27.640182,23.680928,59.008821,58.490141,82.962158,85.511957
std,12.121554,28.000828,21.088363,26.332690,28.650385,18.152709,14.596907
min,1980.000000,0.000000,0.000000,0.920000,1.000000,9.000000,21.970000
25%,1990.000000,4.770000,7.100000,36.235000,31.437500,79.262193,81.980000
50%,2000.500000,19.320000,17.440000,63.359400,65.978846,89.488163,90.300000
75%,2011.000000,41.380000,35.470000,82.195000,83.752500,94.800000,95.250000
max,2021.000000,152.660000,149.410000,100.000000,100.000000,100.000000,99.990000


In [65]:
#saves the data with region column
df.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels_cleanded_regions.csv',index=False,encoding='latin-1')

In [66]:
df_clean_no_regions = df.drop(columns=['Region'])

In [69]:
#save the data cleanly in cleaned file
df_clean_no_regions.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/gender-gap-education-levels_cleanded.csv',index=False,encoding='latin-1')